# VWAP - Tick Forecast

# Debug: update quantity_behind after placing order aggressively
Set tick factor coefficient to 1. Add some logs to the algo_loop, we find that at bar $9$, after placing order aggressively, the quantity_behind will not update, which causes the endless aggressive trading until the total shares are sold out. Thus, we add a line to update the quantity_behind to eliminate the bug.

During the class, professor set the tick factor to 0 and displayed that it would perfectly follow the VWAP. The reason is if we ignore the tick factor, we will never place order aggressively in our loop (the fair value is always above bid_price), given the specific TSLA TAQ data set.
# Exact number of shares traded
After fixing bug, the total shares traded is slightly less than our target $30000$. The reason is at the last bar we trade passively, and our orders are not guaranteed to be fufilled. In order to finish all the orders, at the last bar, we force it to trade aggressively, which achieve our target more precisely but slightly increase the trading cost.

Another thing that prevents us from trading exact $30000$ shares is that we can only trade mutiple of 100 shares when trading aggressively (round_lot=100). We can not change this because it's determined by the exchange. 

## Pick the best Schedule Factor Coefficient

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle

# load our helper functios
import simtools as st
import vwap2_limit_revised as vw

matplotlib.rcParams[ 'figure.figsize' ] = ( 16, 9 )

In [2]:
vwap_coefs = [ 9.95503275e-03, -1.01533564e-04,  5.73879785e-07, -1.47027197e-09, 1.38993793e-12]

In [3]:
# set up some variables
tickfile = 'TSLA-TAQ-jan20.zip'
taq = pd.read_pickle(tickfile)

def filter(df):
    return df[(pd.isna(df.trade_px) == 0) |((pd.isna(df.trade_px) == 1) & ( df.qu_source == 'N') & (df.natbbo_ind == 4 ))]

day1 = taq['2020-01-15']
day1 = filter(day1)
day2 = taq['2020-01-16']
day2 = filter(day2)
day3 = taq['2020-01-17']
day3 = filter(day3)

### Sell

In [4]:
schedule_coefs = np.linspace(0, 1, 20)
avg_slipperage = []
order_quantity = 300000
order_side = 's'

if order_side == 's':
    schedule_coefs = -schedule_coefs
for i, schedule_coef in enumerate(schedule_coefs):
    results1 = vw.algo_loop(day1, order_side, order_quantity, vwap_coefs, schedule_coef)
    results2 = vw.algo_loop(day2, order_side, order_quantity, vwap_coefs, schedule_coef)
    results3 = vw.algo_loop(day3, order_side, order_quantity, vwap_coefs, schedule_coef)      
    
    slippage1 = results1['day_vwap'] - results1['avg_price']
    slippage2 = results2['day_vwap'] - results2['avg_price']
    slippage3 = results3['day_vwap'] - results3['avg_price']
    print("slipages", slippage1, slippage2, slippage3)
    print("If sell, then slippage smaller better")
    print("If buy, then slippage larger better")
    
    avg_slipperage.append((slippage1 + slippage2 + slippage3)/3)

if order_side == 's':   
    best_schedule_coef = schedule_coefs[np.argmin(avg_slipperage)]
else:
    best_schedule_coef = schedule_coefs_s[np.argmax(avg_slipperage)]

2020-03-26 22:03:05:373088: Beginning VWAP run: s 300000 shares
2020-03-26 22:03:05:373282: schedule_coef: -0.0, tick_coef: 0
Average stock spread for sample: 0.2682
2020-03-26 22:03:05:479115: starting main loop
2020-03-26 22:04:39:547114: end simulation loop
2020-03-26 22:04:39:547228: order analytics
2020-03-26 22:04:39:584124: Quantity Remaining: 0
2020-03-26 22:04:39:584232: VWAP run complete.
2020-03-26 22:04:39:612671: Beginning VWAP run: s 300000 shares
2020-03-26 22:04:39:612769: schedule_coef: -0.0, tick_coef: 0
Average stock spread for sample: 0.2473
2020-03-26 22:04:39:675754: starting main loop
2020-03-26 22:06:32:819020: end simulation loop
2020-03-26 22:06:32:819122: order analytics
2020-03-26 22:06:32:855690: Quantity Remaining: 0
2020-03-26 22:06:32:855773: VWAP run complete.
2020-03-26 22:06:32:887649: Beginning VWAP run: s 300000 shares
2020-03-26 22:06:32:887744: schedule_coef: -0.0, tick_coef: 0
Average stock spread for sample: 0.2104
2020-03-26 22:06:32:929824: st

2020-03-26 22:34:01:772841: end simulation loop
2020-03-26 22:34:01:772943: order analytics
2020-03-26 22:34:01:804846: Quantity Remaining: 0
2020-03-26 22:34:01:804939: VWAP run complete.
2020-03-26 22:34:01:834536: Beginning VWAP run: s 300000 shares
2020-03-26 22:34:01:834714: schedule_coef: -0.3157894736842105, tick_coef: 0
Average stock spread for sample: 0.2473
2020-03-26 22:34:01:901477: starting main loop
2020-03-26 22:35:57:430572: end simulation loop
2020-03-26 22:35:57:430671: order analytics
2020-03-26 22:35:57:469427: Quantity Remaining: 0
2020-03-26 22:35:57:469507: VWAP run complete.
2020-03-26 22:35:57:503858: Beginning VWAP run: s 300000 shares
2020-03-26 22:35:57:504024: schedule_coef: -0.3157894736842105, tick_coef: 0
Average stock spread for sample: 0.2104
2020-03-26 22:35:57:546726: starting main loop
2020-03-26 22:37:10:919280: end simulation loop
2020-03-26 22:37:10:919378: order analytics
2020-03-26 22:37:10:943767: Quantity Remaining: 0
2020-03-26 22:37:10:9438

2020-03-26 23:01:24:494689: end simulation loop
2020-03-26 23:01:24:494791: order analytics
2020-03-26 23:01:24:528733: Quantity Remaining: 0
2020-03-26 23:01:24:528819: VWAP run complete.
2020-03-26 23:01:24:555601: Beginning VWAP run: s 300000 shares
2020-03-26 23:01:24:555696: schedule_coef: -0.631578947368421, tick_coef: 0
Average stock spread for sample: 0.2473
2020-03-26 23:01:24:621254: starting main loop
2020-03-26 23:03:13:440473: end simulation loop
2020-03-26 23:03:13:440575: order analytics
2020-03-26 23:03:13:480287: Quantity Remaining: 0
2020-03-26 23:03:13:480373: VWAP run complete.
2020-03-26 23:03:13:513412: Beginning VWAP run: s 300000 shares
2020-03-26 23:03:13:513528: schedule_coef: -0.631578947368421, tick_coef: 0
Average stock spread for sample: 0.2104
2020-03-26 23:03:13:556534: starting main loop
2020-03-26 23:04:20:447003: end simulation loop
2020-03-26 23:04:20:447101: order analytics
2020-03-26 23:04:20:473179: Quantity Remaining: 0
2020-03-26 23:04:20:473265

2020-03-26 23:27:44:304192: end simulation loop
2020-03-26 23:27:44:304291: order analytics
2020-03-26 23:27:44:337875: Quantity Remaining: 0
2020-03-26 23:27:44:337960: VWAP run complete.
2020-03-26 23:27:44:366086: Beginning VWAP run: s 300000 shares
2020-03-26 23:27:44:366181: schedule_coef: -0.9473684210526315, tick_coef: 0
Average stock spread for sample: 0.2473
2020-03-26 23:27:44:432060: starting main loop
2020-03-26 23:29:28:423054: end simulation loop
2020-03-26 23:29:28:423183: order analytics
2020-03-26 23:29:28:466449: Quantity Remaining: 0
2020-03-26 23:29:28:466555: VWAP run complete.
2020-03-26 23:29:28:505869: Beginning VWAP run: s 300000 shares
2020-03-26 23:29:28:505992: schedule_coef: -0.9473684210526315, tick_coef: 0
Average stock spread for sample: 0.2104
2020-03-26 23:29:28:549559: starting main loop
2020-03-26 23:30:47:552143: end simulation loop
2020-03-26 23:30:47:552254: order analytics
2020-03-26 23:30:47:577513: Quantity Remaining: 0
2020-03-26 23:30:47:5776

In [5]:
pickle.dump(schedule_coefs, open("schudule_coefs_s.p", "wb" ))
pickle.dump(avg_slipperage, open("schudule_coefs_avg_slipperage_s.p", "wb" ))
print(best_schedule_coef)
pickle.dump(best_schedule_coef, open("schudule_coefs_s_best.p", "wb" ))

-0.5789473684210527


### Buy

In [6]:
schedule_coefs = np.linspace(0, 1, 20)
avg_slipperage = []
order_quantity = 300000
order_side = 'b'

if order_side == 's':
    schedule_coefs = -schedule_coefs

for i, schedule_coef in enumerate(schedule_coefs):
    results1 = vw.algo_loop(day1, order_side, order_quantity, vwap_coefs, schedule_coef)
    results2 = vw.algo_loop(day2, order_side, order_quantity, vwap_coefs, schedule_coef)
    results3 = vw.algo_loop(day3, order_side, order_quantity, vwap_coefs, schedule_coef)     
    
    slippage1 = results1['day_vwap'] - results1['avg_price']
    slippage2 = results2['day_vwap'] - results2['avg_price']
    slippage3 = results3['day_vwap'] - results3['avg_price']
    print("slipages", slippage1, slippage2, slippage3)
    print("If sell, then slippage smaller better")
    print("If buy, then slippage larger better")
    
    avg_slipperage.append((slippage1 + slippage2 + slippage3)/3)

if order_side == 's':   
    best_schedule_coef = schedule_coefs[np.argmin(avg_slipperage)]
else:
    best_schedule_coef = schedule_coefs[np.argmax(avg_slipperage)]

2020-03-26 23:35:03:455640: Beginning VWAP run: b 300000 shares
2020-03-26 23:35:03:455828: schedule_coef: 0.0, tick_coef: 0
Average stock spread for sample: 0.2682
2020-03-26 23:35:03:518471: starting main loop
2020-03-26 23:36:36:414578: end simulation loop
2020-03-26 23:36:36:414698: order analytics
2020-03-26 23:36:36:448127: Quantity Remaining: 0
2020-03-26 23:36:36:448232: VWAP run complete.
2020-03-26 23:36:36:475310: Beginning VWAP run: b 300000 shares
2020-03-26 23:36:36:475575: schedule_coef: 0.0, tick_coef: 0
Average stock spread for sample: 0.2473
2020-03-26 23:36:36:541334: starting main loop
2020-03-26 23:38:36:110387: end simulation loop
2020-03-26 23:38:36:110513: order analytics
2020-03-26 23:38:36:150980: Quantity Remaining: 0
2020-03-26 23:38:36:151068: VWAP run complete.
2020-03-26 23:38:36:187019: Beginning VWAP run: b 300000 shares
2020-03-26 23:38:36:187271: schedule_coef: 0.0, tick_coef: 0
Average stock spread for sample: 0.2104
2020-03-26 23:38:36:230370: start

2020-03-27 00:05:44:143505: end simulation loop
2020-03-27 00:05:44:143611: order analytics
2020-03-27 00:05:44:176888: Quantity Remaining: 0
2020-03-27 00:05:44:176982: VWAP run complete.
2020-03-27 00:05:44:207274: Beginning VWAP run: b 300000 shares
2020-03-27 00:05:44:207380: schedule_coef: 0.3157894736842105, tick_coef: 0
Average stock spread for sample: 0.2473
2020-03-27 00:05:44:275357: starting main loop
2020-03-27 00:07:44:702444: end simulation loop
2020-03-27 00:07:44:702549: order analytics
2020-03-27 00:07:44:741679: Quantity Remaining: 0
2020-03-27 00:07:44:741775: VWAP run complete.
2020-03-27 00:07:44:775558: Beginning VWAP run: b 300000 shares
2020-03-27 00:07:44:775841: schedule_coef: 0.3157894736842105, tick_coef: 0
Average stock spread for sample: 0.2104
2020-03-27 00:07:44:818665: starting main loop
2020-03-27 00:08:54:840871: end simulation loop
2020-03-27 00:08:54:840976: order analytics
2020-03-27 00:08:54:866306: Quantity Remaining: 0
2020-03-27 00:08:54:866399

2020-03-27 00:35:18:203650: end simulation loop
2020-03-27 00:35:18:203752: order analytics
2020-03-27 00:35:18:236543: Quantity Remaining: 0
2020-03-27 00:35:18:236631: VWAP run complete.
2020-03-27 00:35:18:264196: Beginning VWAP run: b 300000 shares
2020-03-27 00:35:18:264392: schedule_coef: 0.631578947368421, tick_coef: 0
Average stock spread for sample: 0.2473
2020-03-27 00:35:18:331022: starting main loop
2020-03-27 00:37:25:687561: end simulation loop
2020-03-27 00:37:25:687665: order analytics
2020-03-27 00:37:25:725954: Quantity Remaining: 0
2020-03-27 00:37:25:726044: VWAP run complete.
2020-03-27 00:37:25:760182: Beginning VWAP run: b 300000 shares
2020-03-27 00:37:25:760408: schedule_coef: 0.631578947368421, tick_coef: 0
Average stock spread for sample: 0.2104
2020-03-27 00:37:25:804679: starting main loop
2020-03-27 00:38:45:136169: end simulation loop
2020-03-27 00:38:45:136273: order analytics
2020-03-27 00:38:45:162338: Quantity Remaining: 0
2020-03-27 00:38:45:162425: 

2020-03-27 01:02:51:359212: end simulation loop
2020-03-27 01:02:51:359321: order analytics
2020-03-27 01:02:51:391616: Quantity Remaining: 0
2020-03-27 01:02:51:391698: VWAP run complete.
2020-03-27 01:02:51:419632: Beginning VWAP run: b 300000 shares
2020-03-27 01:02:51:419743: schedule_coef: 0.9473684210526315, tick_coef: 0
Average stock spread for sample: 0.2473
2020-03-27 01:02:51:488335: starting main loop
2020-03-27 01:04:44:865046: end simulation loop
2020-03-27 01:04:44:865168: order analytics
2020-03-27 01:04:44:901154: Quantity Remaining: 0
2020-03-27 01:04:44:901240: VWAP run complete.
2020-03-27 01:04:44:937703: Beginning VWAP run: b 300000 shares
2020-03-27 01:04:44:937826: schedule_coef: 0.9473684210526315, tick_coef: 0
Average stock spread for sample: 0.2104
2020-03-27 01:04:44:983086: starting main loop
2020-03-27 01:05:49:746589: end simulation loop
2020-03-27 01:05:49:746690: order analytics
2020-03-27 01:05:49:772471: Quantity Remaining: 0
2020-03-27 01:05:49:772556

In [ ]:
pickle.dump(schedule_coefs, open("schudule_coefs_b.p", "wb" ))
pickle.dump(avg_slipperage, open("schudule_coefs_avg_slipperage_b.p", "wb" ))
print(best_schedule_coef)
pickle.dump(best_schedule_coef, open("schudule_coefs_b_best.p", "wb" ))

0.8421052631578947


## Pick the best Tick Factor Coefficient

### Sell

In [12]:
#这里统一一下吧定下schedule factor coef 优化tick factor coef
avg_slipperage = []
tick_coefs = np.linspace(0, 1, 20)
order_quantity = 300000
order_side = 's'

file = open("schudule_coefs_s_best.p",'rb')
best_schedule_coef = pickle.load(file)
file.close()

if order_side == 's':
    tick_coefs = -tick_coefs
for i, tick_coef in enumerate(tick_coefs):
    results1 = vw.algo_loop(day1, order_side, order_quantity, vwap_coefs, best_schedule_coef, tick_coef)
    results2 = vw.algo_loop(day2, order_side, order_quantity, vwap_coefs, best_schedule_coef, tick_coef)
    results3 = vw.algo_loop(day3, order_side, order_quantity, vwap_coefs, best_schedule_coef, tick_coef)
    
    slippage1 = results1['day_vwap'] - results1['avg_price']
    slippage2 = results2['day_vwap'] - results2['avg_price']
    slippage3 = results3['day_vwap'] - results3['avg_price']
    print("slipages", slippage1, slippage2, slippage3)
    print("If sell, then slippage smaller better")
    print("If buy, then slippage larger better")
    
    avg_slipperage.append((slippage1 + slippage2 + slippage3)/3)

if order_side == 's':   
    best_tick_coef = tick_coefs[np.argmin(avg_slipperage)]
else:
    best_tick_coef = ticke_coefs[np.argmax(avg_slipperage)]

2020-03-27 14:08:56:152172: Beginning VWAP run: s 300000 shares
2020-03-27 14:08:56:152280: schedule_coef: -0.5789473684210527, tick_coef: -0.0
Average stock spread for sample: 0.2682
2020-03-27 14:08:56:209280: starting main loop
2020-03-27 14:10:41:564958: end simulation loop
2020-03-27 14:10:41:565100: order analytics
2020-03-27 14:10:41:602425: Quantity Remaining: 0
2020-03-27 14:10:41:602515: VWAP run complete.
2020-03-27 14:10:41:634376: Beginning VWAP run: s 300000 shares
2020-03-27 14:10:41:634684: schedule_coef: -0.5789473684210527, tick_coef: -0.0
Average stock spread for sample: 0.2473
2020-03-27 14:10:41:708014: starting main loop
2020-03-27 14:12:39:640452: end simulation loop
2020-03-27 14:12:39:640557: order analytics
2020-03-27 14:12:39:681924: Quantity Remaining: 0
2020-03-27 14:12:39:682013: VWAP run complete.
2020-03-27 14:12:39:717895: Beginning VWAP run: s 300000 shares
2020-03-27 14:12:39:718116: schedule_coef: -0.5789473684210527, tick_coef: -0.0
Average stock sp

2020-03-27 14:36:26:029992: end simulation loop
2020-03-27 14:36:26:030116: order analytics
2020-03-27 14:36:26:054580: Quantity Remaining: 0
2020-03-27 14:36:26:054684: VWAP run complete.
slipages -0.7176137824716307 -0.00779437537113381 0.1816959057441636
If sell, then slippage smaller better
If buy, then slippage larger better
2020-03-27 14:36:26:075354: Beginning VWAP run: s 300000 shares
2020-03-27 14:36:26:075413: schedule_coef: -0.5789473684210527, tick_coef: -0.3157894736842105
Average stock spread for sample: 0.2682
2020-03-27 14:36:26:129209: starting main loop
2020-03-27 14:37:52:645741: end simulation loop
2020-03-27 14:37:52:645878: order analytics
2020-03-27 14:37:52:680889: Quantity Remaining: 0
2020-03-27 14:37:52:680976: VWAP run complete.
2020-03-27 14:37:52:709914: Beginning VWAP run: s 300000 shares
2020-03-27 14:37:52:710012: schedule_coef: -0.5789473684210527, tick_coef: -0.3157894736842105
Average stock spread for sample: 0.2473
2020-03-27 14:37:52:773460: starti

2020-03-27 15:01:39:155665: end simulation loop
2020-03-27 15:01:39:155770: order analytics
2020-03-27 15:01:39:195600: Quantity Remaining: 0
2020-03-27 15:01:39:195736: VWAP run complete.
2020-03-27 15:01:39:233305: Beginning VWAP run: s 300000 shares
2020-03-27 15:01:39:233523: schedule_coef: -0.5789473684210527, tick_coef: -0.5789473684210527
Average stock spread for sample: 0.2104
2020-03-27 15:01:39:281386: starting main loop
2020-03-27 15:02:42:884537: end simulation loop
2020-03-27 15:02:42:884644: order analytics
2020-03-27 15:02:42:909085: Quantity Remaining: 0
2020-03-27 15:02:42:909186: VWAP run complete.
slipages -0.7157378972061679 -0.0036646862572524697 0.18320884664819914
If sell, then slippage smaller better
If buy, then slippage larger better
2020-03-27 15:02:42:931157: Beginning VWAP run: s 300000 shares
2020-03-27 15:02:42:931318: schedule_coef: -0.5789473684210527, tick_coef: -0.631578947368421
Average stock spread for sample: 0.2682
2020-03-27 15:02:42:985856: star

2020-03-27 15:25:11:937742: end simulation loop
2020-03-27 15:25:11:937841: order analytics
2020-03-27 15:25:11:967766: Quantity Remaining: 0
2020-03-27 15:25:11:967850: VWAP run complete.
2020-03-27 15:25:11:997727: Beginning VWAP run: s 300000 shares
2020-03-27 15:25:11:997826: schedule_coef: -0.5789473684210527, tick_coef: -0.894736842105263
Average stock spread for sample: 0.2473
2020-03-27 15:25:12:064115: starting main loop
2020-03-27 15:26:55:145617: end simulation loop
2020-03-27 15:26:55:145784: order analytics
2020-03-27 15:26:55:204717: Quantity Remaining: 0
2020-03-27 15:26:55:204845: VWAP run complete.
2020-03-27 15:26:55:336391: Beginning VWAP run: s 300000 shares
2020-03-27 15:26:55:336524: schedule_coef: -0.5789473684210527, tick_coef: -0.894736842105263
Average stock spread for sample: 0.2104
2020-03-27 15:26:55:406373: starting main loop
2020-03-27 15:28:35:940527: end simulation loop
2020-03-27 15:28:35:950118: order analytics
2020-03-27 15:28:36:271136: Quantity Rem

In [13]:
pickle.dump(schedule_coefs, open("tick_coefs_s.p", "wb" ))
pickle.dump(avg_slipperage, open("tick_coefs_avg_slipperage_s.p", "wb" ))
print(best_tick_coef)
pickle.dump(best_tick_coef, open("tick_coefs_s_best.p", "wb" ))

-0.10526315789473684


### Buy

In [14]:
#这里统一一下吧定下schedule factor coef 优化tick factor coef
avg_slipperage = []
tick_coefs = np.linspace(0, 1, 20)
order_quantity = 300000
order_side = 'b'

file = open("schudule_coefs_b_best.p",'rb')
best_schedule_coef = pickle.load(file)
file.close()

if order_side == 's':
    tick_coefs = -tick_coefs
for i, tick_coef in enumerate(tick_coefs):
    results1 = vw.algo_loop(day1, order_side, order_quantity, vwap_coefs, best_schedule_coef, tick_coef)
    results2 = vw.algo_loop(day2, order_side, order_quantity, vwap_coefs, best_schedule_coef, tick_coef)
    results3 = vw.algo_loop(day3, order_side, order_quantity, vwap_coefs, best_schedule_coef, tick_coef)
    
    slippage1 = results1['day_vwap'] - results1['avg_price']
    slippage2 = results2['day_vwap'] - results2['avg_price']
    slippage3 = results3['day_vwap'] - results3['avg_price']
    print("slipages", slippage1, slippage2, slippage3)
    print("If sell, then slippage smaller better")
    print("If buy, then slippage larger better")
    
    avg_slipperage.append((slippage1 + slippage2 + slippage3)/3)

if order_side == 's':   
    best_tick_coef = tick_coefs[np.argmin(avg_slipperage)]
else:
    best_tick_coef = tick_coefs[np.argmax(avg_slipperage)]

2020-03-27 15:37:07:878083: Beginning VWAP run: b 300000 shares
2020-03-27 15:37:07:878367: schedule_coef: 0.8421052631578947, tick_coef: 0.0
Average stock spread for sample: 0.2682
2020-03-27 15:37:07:934576: starting main loop
2020-03-27 15:38:35:986891: end simulation loop
2020-03-27 15:38:35:986996: order analytics
2020-03-27 15:38:36:021301: Quantity Remaining: 0
2020-03-27 15:38:36:021385: VWAP run complete.
2020-03-27 15:38:36:048272: Beginning VWAP run: b 300000 shares
2020-03-27 15:38:36:048385: schedule_coef: 0.8421052631578947, tick_coef: 0.0
Average stock spread for sample: 0.2473
2020-03-27 15:38:36:114548: starting main loop
2020-03-27 15:40:22:864392: end simulation loop
2020-03-27 15:40:22:864499: order analytics
2020-03-27 15:40:22:904836: Quantity Remaining: 0
2020-03-27 15:40:22:904922: VWAP run complete.
2020-03-27 15:40:22:938592: Beginning VWAP run: b 300000 shares
2020-03-27 15:40:22:938709: schedule_coef: 0.8421052631578947, tick_coef: 0.0
Average stock spread f

2020-03-27 16:06:34:646162: end simulation loop
2020-03-27 16:06:34:646258: order analytics
2020-03-27 16:06:34:670256: Quantity Remaining: 0
2020-03-27 16:06:34:670336: VWAP run complete.
slipages -0.7738582051803178 -0.06567363751042876 0.1405697012849032
If sell, then slippage smaller better
If buy, then slippage larger better
2020-03-27 16:06:34:691905: Beginning VWAP run: b 300000 shares
2020-03-27 16:06:34:691964: schedule_coef: 0.8421052631578947, tick_coef: 0.3157894736842105
Average stock spread for sample: 0.2682
2020-03-27 16:06:34:749813: starting main loop
2020-03-27 16:08:03:958963: end simulation loop
2020-03-27 16:08:03:959067: order analytics
2020-03-27 16:08:03:991960: Quantity Remaining: 0
2020-03-27 16:08:03:992043: VWAP run complete.
2020-03-27 16:08:04:021465: Beginning VWAP run: b 300000 shares
2020-03-27 16:08:04:021581: schedule_coef: 0.8421052631578947, tick_coef: 0.3157894736842105
Average stock spread for sample: 0.2473
2020-03-27 16:08:04:089313: starting m

2020-03-27 16:31:35:544473: end simulation loop
2020-03-27 16:31:35:544575: order analytics
2020-03-27 16:31:35:581925: Quantity Remaining: 0
2020-03-27 16:31:35:582082: VWAP run complete.
2020-03-27 16:31:35:616212: Beginning VWAP run: b 300000 shares
2020-03-27 16:31:35:616311: schedule_coef: 0.8421052631578947, tick_coef: 0.5789473684210527
Average stock spread for sample: 0.2104
2020-03-27 16:31:35:659003: starting main loop
2020-03-27 16:32:40:042556: end simulation loop
2020-03-27 16:32:40:042657: order analytics
2020-03-27 16:32:40:068920: Quantity Remaining: 0
2020-03-27 16:32:40:069006: VWAP run complete.
slipages -0.7728006613000389 -0.05938371268450737 0.15385473461827814
If sell, then slippage smaller better
If buy, then slippage larger better
2020-03-27 16:32:40:090356: Beginning VWAP run: b 300000 shares
2020-03-27 16:32:40:090413: schedule_coef: 0.8421052631578947, tick_coef: 0.631578947368421
Average stock spread for sample: 0.2682
2020-03-27 16:32:40:146805: starting m

2020-03-27 16:54:05:510136: end simulation loop
2020-03-27 16:54:05:510254: order analytics
2020-03-27 16:54:05:542219: Quantity Remaining: 0
2020-03-27 16:54:05:542307: VWAP run complete.
2020-03-27 16:54:05:572090: Beginning VWAP run: b 300000 shares
2020-03-27 16:54:05:572192: schedule_coef: 0.8421052631578947, tick_coef: 0.894736842105263
Average stock spread for sample: 0.2473
2020-03-27 16:54:05:639038: starting main loop
2020-03-27 16:55:42:557664: end simulation loop
2020-03-27 16:55:42:557762: order analytics
2020-03-27 16:55:42:595513: Quantity Remaining: 0
2020-03-27 16:55:42:595599: VWAP run complete.
2020-03-27 16:55:42:633426: Beginning VWAP run: b 300000 shares
2020-03-27 16:55:42:633688: schedule_coef: 0.8421052631578947, tick_coef: 0.894736842105263
Average stock spread for sample: 0.2104
2020-03-27 16:55:42:676364: starting main loop
2020-03-27 16:56:39:943645: end simulation loop
2020-03-27 16:56:39:943746: order analytics
2020-03-27 16:56:39:967408: Quantity Remaini

In [15]:
if order_side == 's':   
    best_tick_coef = tick_coefs[np.argmin(avg_slipperage)]
else:
    best_tick_coef = tick_coefs[np.argmax(avg_slipperage)]

pickle.dump(schedule_coefs, open("tick_coefs_b.p", "wb" ))
pickle.dump(avg_slipperage, open("tick_coefs_avg_slipperage_b.p", "wb" ))
print(best_tick_coef)
pickle.dump(best_tick_coef, open("tick_coefs_b_best.p", "wb" ))

1.0
